# CS 178 Project
## Import part

In [22]:
import os
import numpy as np
import pandas as pd
import cv2

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Read image for train

In [31]:
data = pd.read_csv('data/500_picts_satz.csv')


image_paths = data.iloc[:, 1]
labels = data.iloc[:, 2]

folder_path = 'images/'

X = []
y = labels

# # Comment below when training
# size = 20
# count = 0
# # Comment above when training

for filename in image_paths:
    # # Comment below when training
    # if count >= size:
    #     break
    # # Comment above when training
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for common image file extensions
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img - cv2.resize(img, (350, 350))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img is not None:
            X.append(torch.Tensor(img))
    count += 1
X = torch.stack(X).unsqueeze(1) # convert to pytorch tensor

# # Comment below when training
# for image, label in zip(X, y):
#     plt.imshow(image.numpy()[0])
#     plt.title(label)
#     plt.show()
# # Comment above when training

## CNN Test
### Define a Convolutional Neural Network

In [33]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) # 1 because its greyscale
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 84 * 84, 120) # shape of image after pooling twice
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7) # 7 is label count

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
Xtr, Ytr = X[:300], y[:300]
Xva, Yva = X[300:500], y[300:500]

In [34]:
F.softmax(net(Xtr), dim=0)

tensor([[0.0022, 0.0045, 0.0033,  ..., 0.0043, 0.0024, 0.0022],
        [0.0028, 0.0027, 0.0039,  ..., 0.0018, 0.0031, 0.0024],
        [0.0031, 0.0035, 0.0023,  ..., 0.0035, 0.0031, 0.0034],
        ...,
        [0.0074, 0.0043, 0.0022,  ..., 0.0052, 0.0093, 0.0046],
        [0.0076, 0.0014, 0.0018,  ..., 0.0082, 0.0036, 0.0020],
        [0.0043, 0.0024, 0.0027,  ..., 0.0070, 0.0018, 0.0063]],
       grad_fn=<SoftmaxBackward0>)

In [35]:
Xtr.shape

torch.Size([300, 1, 350, 350])

### Define a Loss function and optimizer

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Train the network

In [37]:
batch_size = 4
trainloader = torch.utils.data.DataLoader(X, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(y, batch_size=batch_size, shuffle=False, num_workers=2)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(zip(Xtr, ytr), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x7056 and 112896x120)